In [ ]:
# ◉_◉

In [ ]:
# for doc, score in query_response:
#     print("*" * 40)
#     print(doc.dict().keys())
#     # print(doc.from_orm(doc).dict())
#     print("score: %s" % score)
#     print(doc.page_content)
#     print("*" * 40)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.prompts import PromptTemplate
# Set logging for the queries
import logging

In [ ]:
# TODO: Clean PDFS, is everything relevant?
query = """What are some key take aways from the agent tuning to get me started? Lets go through it step by step, and help me devise a plan for implementing some of the ideas in python code."""

In [ ]:
llm = ChatOpenAI(temperature=0.44,openai_api_key=oai_sk)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=db_conn.as_retriever(), llm=llm
)

In [ ]:
unique_docs = retriever_from_llm.get_relevant_documents(query=query)

In [ ]:
unique_docs[0].dict().keys()
unique_docs[0].metadata

In [ ]:
proompt = """Use the following bits of information and internalize them. Because, you will then be applying the knowledge in a step by step manner to help a research agent in formulating a plan to implement the ideas in python code. Remember to be concise and pragmatic in your answer, and also focus on changing the most specific thing for the most specific reason. Remember, this means a lot to us, experimentation and research are important and you are good at both.

Here is the relevant information:
{env_context}

And this is the current task at hand:
{task_description}

Your answer:"""

big_proompt = PromptTemplate(
    template=proompt, input_variables=["env_context", "task_description"]
)

In [ ]:
response = llm.predict(
    text=big_proompt.format_prompt(env_context=unique_docs, task_description=query).text
)

In [ ]:
from utils.utils import extract_functions_from_repo
from pathlib import Path
codedir = Path.cwd() / "temprepo"
all_functions = extract_functions_from_repo(codedir)

In [ ]:
all_functions

In [ ]:
from embeddings import create_embedding_collection, load_and_chunk_code, get_embedding_func
import openai
import langchain
# db_conn = get_db_connection("agentTuning_code_vecdb")

In [ ]:
split_code_docs = load_and_chunk_code("./temprepo/")
split_code_docs
code_embedding_dbconn = create_embedding_collection(
    chunked_docs=split_code_docs, embeddings=get_embedding_func(),
    collection_name="agentTuning_code_vecdb"
)

In [ ]:
for doc in code_embedding_dbconn.similarity_search("mmlu eval"):
    print(doc.page_content)
# print(code_embedding_dbconn.similarity_search("mmlu eval")[0].page_content)